In [75]:
import pandas as pd
import numpy as np
import unidecode
from stop_words import get_stop_words

In [3]:
precios = pd.read_pickle('precios.pkl')
productos = pd.read_pickle('productos.pkl')
sucursales = pd.read_pickle('sucursales.pkl')

In [4]:
precios.head()

,_id,fecha,medicion,precio,producto,sucursal
0,5cba50f27af1523c35613043,2019-01-15 04:51:28.220,6,76.99,12-1-2800000937881,12-1-44
1,5cba50f27af1523c3561307b,2019-01-15 04:51:28.220,6,92.87,7790240041880,12-1-44
2,5cba50f27af1523c35613042,2019-01-15 04:51:28.220,6,56.20,7790762052364,12-1-44
3,5cba50f27af1523c3561307a,2019-01-15 04:51:28.220,6,215.00,7793440000305,12-1-44
4,5cba50f27af1523c35613082,2019-01-15 04:51:28.220,6,81.99,7790040999503,12-1-44


In [5]:
productos.head()

,_id,id,marca,nombre,presentacion
0,5cbc69be7af152186c0cd485,0000075027513,DOVE,Desodorante Original Dove 50 Gr,50.0 gr
1,5cbc69be7af152186c0cd488,0000077940131,ARCOR,Turron Arcor 25 Gr,25.0 gr
2,5cbc69be7af152186c0cd489,0000077940704,REXONA,Desodorante en Crema en Pote Odorono Rexona 60 Gr,60.0 gr
3,5cbc69be7af152186c0cd48a,0000077953063,DANETTE,Postre Dulce de Leche Danette Pack 2 Un 190 Gr,190.0 gr
4,5cbc69be7af152186c0cd487,0000077903518,OPERA,Galletitas Obleas Rellena Opera 92 Gr,92.0 gr


In [6]:
sucursales.head()

,_id,banderaDescripcion,banderaId,comercioId,comercioRazonSocial,direccion,id,lat,lng,localidad,provincia,sucursalId,sucursalNombre,sucursalTipo
0,5cbc698b7af152186c0cd142,DEHEZA S.A.I.C.F. e I.,1,3,Deheza S.A.I.C.F. e I.,Av. General Paz 3802,3-1-1507,-34.559976,-58.504540,CAPITAL FEDERAL,AR-C,1507,GRAL PAZ - SUR,Autoservicio
1,5cbc698b7af152186c0cd141,Express,3,10,INC S.A.,Av. Cabildo 4861,10-3-675,-34.540038,-58.474738,Ciudad Autónoma de Buenos Aires,AR-C,675,Cabildo 4861,Autoservicio
2,5cbc698b7af152186c0cd140,DEHEZA S.A.I.C.F. e I.,1,3,Deheza S.A.I.C.F. e I.,San Juan Bautista De La Salle 4356,3-1-1506,-34.559454,-58.505028,CAPITAL FEDERAL,AR-C,1506,GRAL PAZ - NORTE,Autoservicio
3,5cbc698b7af152186c0cd143,DEHEZA S.A.I.C.F. e I.,1,3,Deheza S.A.I.C.F. e I.,Cabildo 4701,3-1-29,-34.541473,-58.473840,CAPITAL FEDERAL,AR-C,29,CABILDO,Autoservicio
4,5cbc698b7af152186c0cd144,Express,3,10,INC S.A.,Gral. Deheza 2239,10-3-300,-34.540539,-58.472052,Ciudad Autónoma de Buenos Aires,AR-C,300,Gral Deheza 2239,Autoservicio


In [7]:
di = {1:1, 2:1, 3:1, 4:2, 5:2, 6:3, 7:3, 8:4, 9:4, 10:4}

In [8]:
precios['periodo'] = precios['medicion'].replace(di)

In [9]:
#pasar a formato columnar
precios2 = precios.pivot_table(index=['producto','sucursal'], columns = 'medicion', values = 'precio')

In [10]:
#rellenar con los precios aledaños
precios2 = precios2.interpolate(axis =1)

In [11]:
precios2.head()

medicion                  1     2     3     4     5      6      7     8   \
producto      sucursal                                                     
0000040084107 10-1-214  36.0  36.0  36.0  36.0  36.0  36.00  43.50  43.5   
              10-1-219  36.0  36.0  36.0  36.0  36.0  36.00  43.50  43.5   
              10-1-220  36.0  36.0  36.0  36.0  36.0  36.00  39.75  43.5   
              10-1-26   36.0  36.0  36.0  36.0  36.0  39.75  43.50  43.5   
              10-1-271  36.0  36.0  36.0  36.0  36.0  36.00  43.50  43.5   

medicion                  9     10  
producto      sucursal              
0000040084107 10-1-214  43.5  55.0  
              10-1-219  46.0  55.0  
              10-1-220  43.5  55.0  
              10-1-26   43.5  55.0  
              10-1-271  46.0  55.0

In [12]:
#columnas por periodo y final
precios2['p1'] = precios2[[1,2,3]].mean(axis=1)
precios2['p2'] = precios2[[4,5]].mean(axis=1)
precios2['p3'] = precios2[[6,7]].mean(axis=1)
precios2['p4'] = precios2[[8,9,10]].mean(axis=1)
precios2['mean'] = precios2.mean(axis=1)

In [13]:
#eliminar los que tengan faltantes en las nuevas columnas
precios3 = precios2.dropna(subset = ['p1', 'p2', 'p3', 'p4', 'mean'])

In [14]:
def var(row, c1, c2):
    res = (row[c2] - row[c1]) / row[c2]
    return(res)

In [15]:
#variacion intra periodo
precios3['var1'] = (precios3.p2 - precios3.p1 ) / precios3.p1
precios3['var2'] = (precios3.p3 - precios3.p2 ) / precios3.p1
precios3['var3'] = (precios3.p4 - precios3.p3 ) / precios3.p1
precios3['var_tot'] = (precios3.p4 - precios3.p1 ) / precios3.p1

/Users/rlopezlam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/rlopezlam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/rlopezlam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [16]:
precios3['var1_cat'] = pd.cut(precios3['var1'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['df', 'dm', 'dl', 'm', 'al', 'am', 'af'])

precios3['var2_cat'] = pd.cut(precios3['var2'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['df', 'dm', 'dl', 'm', 'al', 'am', 'af'])

precios3['var3_cat'] = pd.cut(precios3['var3'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['df', 'dm', 'dl', 'm', 'al', 'am', 'af'])

precios3['var_tot_cat'] = pd.cut(precios3['var_tot'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['df', 'dm', 'dl', 'm', 'al', 'am', 'af'])

/Users/rlopezlam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/rlopezlam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/rlopezlam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [17]:
precios3.head()

medicion                   1     2     3     4     5      6      7     8  \
producto      sucursal                                                     
0000040084107 10-1-214  36.0  36.0  36.0  36.0  36.0  36.00  43.50  43.5   
              10-1-219  36.0  36.0  36.0  36.0  36.0  36.00  43.50  43.5   
              10-1-220  36.0  36.0  36.0  36.0  36.0  36.00  39.75  43.5   
              10-1-26   36.0  36.0  36.0  36.0  36.0  39.75  43.50  43.5   
              10-1-271  36.0  36.0  36.0  36.0  36.0  36.00  43.50  43.5   

medicion                   9    10  ...         p4       mean  var1      var2  \
producto      sucursal              ...                                         
0000040084107 10-1-214  43.5  55.0  ...  47.333333  40.041667   0.0  0.104167   
              10-1-219  46.0  55.0  ...  48.166667  40.279762   0.0  0.104167   
              10-1-220  43.5  55.0  ...  47.333333  39.639881   0.0  0.052083   
              10-1-26   43.5  55.0  ...  47.333333  40.443452   0.0  0.156250   
              10-1-271  46.0  55.0  ...  48.166667  40.279762   0.0  0.104167   

medicion                    var3   var_tot  var1_cat  var2_cat  var3_cat  \
producto      sucursal                                                     
0000040084107 10-1-214  0.210648  0.314815         m        af        af   
              10-1-219  0.233796  0.337963         m        af        af   
              10-1-220  0.262731  0.314815         m        am        af   
              10-1-26   0.158565  0.314815         m        af        af   
              10-1-271  0.233796  0.337963         m        af        af   

medicion               var_tot_cat  
producto      sucursal              
0000040084107 10-1-214          af  
              10-1-219          af  
              10-1-220          af  
              10-1-26           af  
              10-1-271          af  

[5 rows x 23 columns]

In [18]:
precios4 = precios3.reset_index().merge(precios3.groupby('producto')[['p1','p2', 'p3', 'p4']].mean(), 
                         how = 'left',
                         left_on = 'producto',
                         right_on = 'producto')

precios4.set_index(['producto','sucursal'], inplace=True)

In [19]:
precios4['pr1'] = (precios4.p1_x - precios4.p1_y ) / precios4.p1_y
precios4['pr2'] = (precios4.p2_x - precios4.p2_y ) / precios4.p2_y
precios4['pr3'] = (precios4.p3_x - precios4.p3_y ) / precios4.p3_y
precios4['pr4'] = (precios4.p4_x - precios4.p4_y ) / precios4.p4_y
precios4

medicion                     1      2      3      4      5       6      7  \
producto      sucursal                                                      
0000040084107 10-1-214   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-1-219   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-1-220   36.00  36.00  36.00  36.00  36.00  36.000  39.75   
              10-1-26    36.00  36.00  36.00  36.00  36.00  39.750  43.50   
              10-1-271   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-1-3     36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-1-30    36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-1-5     36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-1-6     36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-115   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-116   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-118   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-120   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-148   44.50  44.50  44.50  44.50  44.50  44.500  44.50   
              10-2-150     NaN    NaN  47.00  47.00  47.00  47.000  47.00   
              10-2-151   36.00  36.00  36.00  36.00  36.00  39.750  43.50   
              10-2-182     NaN    NaN  47.00  47.00  47.00  47.000  47.00   
              10-2-189   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-195   44.50  44.50  44.50  44.50  44.50  44.500  44.50   
              10-2-198   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-206   46.10  48.40  47.00  47.00  47.00  47.000  47.00   
              10-2-215   36.00  36.00  36.00  36.00  36.00  39.750  43.50   
              10-2-228   48.40  48.40  47.00  47.00  47.00  47.000  47.00   
              10-2-237   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-258   36.00  36.00  36.00  36.00  36.00  39.750  43.50   
              10-2-277   46.10  48.40  47.00  47.00  47.00  47.000  47.00   
              10-2-381   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-385   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-386   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
              10-2-389   36.00  36.00  36.00  36.00  36.00  36.000  43.50   
...                        ...    ...    ...    ...    ...     ...    ...   
7898422746827 9-2-39     42.10  42.10  42.50  42.50  42.50  43.700  44.90   
              9-2-46     41.75  42.49  42.89  42.89  42.89  44.090  45.29   
              9-2-48     41.75  41.75  42.10  42.10  42.10  43.295  44.49   
              9-2-54     41.35  42.10  42.50  42.50  42.50  43.700  44.90   
              9-2-630    41.75  41.75  42.10  42.10  42.10  42.100  44.49   
              9-2-701    41.35  42.10  42.50  42.50  42.50  42.500  44.90   
              9-2-916    42.49  43.89  42.89  42.89  42.89  42.890  48.25   
              9-2-977    41.75  42.49  42.89  42.89  42.89  46.350  45.29   
              9-3-121    37.65  40.69  39.99  39.99  39.99  41.090  42.19   
              9-3-5201   40.80  42.29  42.29  42.29  42.29  44.620  46.95   
              9-3-5206   41.15  41.15  41.89  41.89  41.89  41.890  41.89   
              9-3-5213   47.00  47.00  47.00  47.00  47.00  47.000  47.00   
              9-3-5231   38.15  38.90  40.35  40.25  40.25  41.850  43.45   
              9-3-5259   39.65  39.65  41.10  41.00  41.00  42.625  44.25   
              9-3-5260   38.85  42.10  43.10  41.00  41.00  45.250  44.25   
              9-3-5263   38.85  39.65  41.10  41.99  41.00  45.250  44.25   
              9-3-5961   41.89  41.89  42.65  42.65  42.65  42.650  48.59   
8480017674449 15-1-1066  47.99  49.99  49.99  49.99  49.99  49.990  49.99   
              15-1-1072  47.99  49.99  49.99  49.99  49.99  49.990  49.99

In [20]:
precios4['pr1_cat'] = pd.cut(precios4['pr1'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['muc', 'mec', 'lec', 'm', 'leb', 'meb', 'mub'])

precios4['pr2_cat'] = pd.cut(precios4['pr2'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['muc', 'mec', 'lec', 'm', 'leb', 'meb', 'mub'])

precios4['pr3_cat'] = pd.cut(precios4['pr3'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['muc', 'mec', 'lec', 'm', 'leb', 'meb', 'mub'])

precios4['pr4_cat'] = pd.cut(precios4['pr4'], 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['muc', 'mec', 'lec', 'm', 'leb', 'meb', 'mub'])

precios4['prm_cat'] = pd.cut(precios4[['pr1','pr2','pr3','pr4']].mean(), 
                              bins=[-1.1, -0.1, -0.05, -0.005, 0.005, 0.05, 0.1, 1.1],
                             labels = ['muc', 'mec', 'lec', 'm', 'leb', 'meb', 'mub'])

In [21]:
precios_final = precios4.reset_index()[['producto','sucursal','pr1_cat', 'pr2_cat', 'pr3_cat', 'pr4_cat',
                                       'prm_cat', 'var1_cat','var2_cat', 'var3_cat', 'var_tot_cat']]

In [68]:
productos1 = productos.copy()
productos1.head()

,_id,id,marca,nombre,presentacion
0,5cbc69be7af152186c0cd485,0000075027513,DOVE,Desodorante Original Dove 50 Gr,50.0 gr
1,5cbc69be7af152186c0cd488,0000077940131,ARCOR,Turron Arcor 25 Gr,25.0 gr
2,5cbc69be7af152186c0cd489,0000077940704,REXONA,Desodorante en Crema en Pote Odorono Rexona 60 Gr,60.0 gr
3,5cbc69be7af152186c0cd48a,0000077953063,DANETTE,Postre Dulce de Leche Danette Pack 2 Un 190 Gr,190.0 gr
4,5cbc69be7af152186c0cd487,0000077903518,OPERA,Galletitas Obleas Rellena Opera 92 Gr,92.0 gr


In [70]:
#minuscula
productos1[['nombre', 'marca', 'presentacion']] = productos1[['nombre', 'marca', 'presentacion']].apply(lambda x: x.str.lower())

# remover numberos
productos1[['nombre', 'marca', 'presentacion']] = productos1[['nombre', 'marca', 'presentacion']].apply(lambda x: x.str.replace('\d+', ''))

# remover puntuacion
productos1[['nombre', 'marca', 'presentacion']] = productos1[['nombre', 'marca', 'presentacion']].apply(lambda x: x.str.replace('[^\w\s]',''))

# remover tildes
productos1['nombre'] = productos1['nombre'].apply(unidecode.unidecode)
productos1['marca'] = productos1['marca'].apply(unidecode.unidecode)
productos1['presentacion'] = productos1['presentacion'].apply(unidecode.unidecode)

#sacar espacios
productos1[['marca', 'presentacion']] = productos1[['marca', 'presentacion']].apply(lambda x: x.str.strip())



In [78]:
marcas = productos1.marca.unique()
presentaciones = productos1.presentacion.unique()
stop_words = get_stop_words('spanish')

def remover(string, stop_words):
    words = string.split()
    resultwords = [word for word in words if word not in stop_words]
    result = ' '.join(resultwords)
    return(result)


productos1['nombre'] = productos1['nombre'].apply(lambda x: remover(x, marcas))
productos1['nombre'] = productos1['nombre'].apply(lambda x: remover(x, presentaciones))
productos1['nombre'] = productos1['nombre'].apply(lambda x: remover(x, stop_words))


productos1.head()

,_id,id,marca,nombre,presentacion
0,5cbc69be7af152186c0cd485,0000075027513,dove,desodorante original,gr
1,5cbc69be7af152186c0cd488,0000077940131,arcor,turron,gr
2,5cbc69be7af152186c0cd489,0000077940704,rexona,desodorante crema pote odorono,gr
3,5cbc69be7af152186c0cd48a,0000077953063,danette,postre dulce leche pack,gr
4,5cbc69be7af152186c0cd487,0000077903518,opera,galletitas obleas rellena,gr


In [64]:
a = ' lo que paso aqui dove'
remover(a, marcas)

'lo que paso aqui'